In [1]:
from Bio.SearchIO import HmmerIO
from Bio import AlignIO, SeqIO
import re 

import pandas as pd
import subprocess

from glob import glob
import os

from tqdm import tqdm
tqdm.pandas()

import pandas as pd
from addict import Dict
import sys
sys.path.append('../03_acetyl')
from funcs1 import *

from matplotlib_venn import venn2

pd.options.display.max_columns = 50

from pandarallel import pandarallel
pandarallel.initialize(nb_workers = 4, progress_bar = True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
# final = pd.read_csv('atp_subunits_operons.csv')
# print(final.shape, final.drop_duplicates().shape)
# final['logevalue'] = final.evalue.apply(lambda x: round(np.log10(x), 2))
# print(final.shape)
# # final.head(2)

In [3]:
# final0[final0.fasta_id == 'AE009951.2_228']

In [4]:
# final['enz']  = None
# final['opt']  = None
# final['enzt'] = None
# final['step'] = None

In [5]:
####Test
# test_g = ['GB_GCA_000007325.1', 'GB_GCA_000210015.1', 'GB_GCA_000431495.1', 'GB_GCA_002452745.1']
# final[final.genome == test_g[3]]

In [6]:
# final.shape[0], final.fasta_id.nunique()

In [7]:
# ! mkdir enzyme_inf_steps/step_opt

In [8]:
# def find_opt(df):
#     genome = df.genome.iloc[0]
#     seq_id = df.seq_id.iloc[0]
#     operon = df.operons.iloc[0]
#     strand = df.strand.unique()
#     if len(strand) > 1:
#         display(df)        
# #     if os.path.isfile(f'enzyme_inf_steps/step_opt/{genome}{seq_id}{operon}'):
# #         return
#     abbr = {'a':'a', 'b': 'b', 'c' :'c', 
#            'alpha':'A', 'beta' : 'B', 'gamma' : 'G', 
#            'delta' : 'D', 'epsilon' : 'E'}
#     df = df.sort_values('start')
#     x  = df.subunit.to_list()[::strand[0]]
#     opt = [abbr[i] for i in x] 
#     opt = ''.join(opt)
# #     final.at[df.index, 'opt'] = opt
#     df['opt'] = opt
    
#     os.makedirs(f'enzyme_inf_steps/step_opt/{genome}', exist_ok = True)
#     df.to_csv(f'enzyme_inf_steps/step_opt/{genome}/{seq_id}{operon}', index = False)
    
# final.groupby(['genome', 'seq_id', 'operons']).progress_apply(find_opt)

In [9]:
# ! ls enzyme_inf_steps/step_opt | wc -l

In [10]:
# ! mkdir enzyme_inf_steps/step_opt1

In [11]:
# for path in tqdm(glob('enzyme_inf_steps/step_opt/*')):
#     df = pd.concat([pd.read_csv(path1, index_col = 0) for path1 in glob(path + '/*')])
#     genome = path.split('/')[-1]
#     df.to_csv(f'enzyme_inf_steps/step_opt1/{genome}', index = False)

In [12]:
final = pd.concat([pd.read_csv(path) for path in 
                   tqdm(glob('enzyme_inf_steps/step_opt1/*'))
                  ])

100%|███████████████████████████████████████████████████████████████████████████████████| 44546/44546 [01:06<00:00, 671.93it/s]


In [13]:
# final = pd.read_csv('enzyme_inf_steps/step_opt1/RS_GCF_903644075.1')

In [14]:
final['enz']  = None
final['enzt'] = None
final['step'] = None

In [15]:
final.head(1)

,start,end,strand,ID,partial,start_type,rbs_motif,rbs_spacer,gc_cont,fasta_id,bitscore,evalue,genome,seq_id,subunit,N,bitscore_mean,c_domains,operons,logevalue,opt,enz,enzt,step
0,2095960,2096274,1,6_1925,0,ATG,AGxAGG/AGGxGG,5-10bp,0.619,NZ_LT629945.1_1925,20.2,0.0001,RS_GCF_900104675.1,NZ_LT629945.1,b,1925,NaN,NaN,NZ_LT629945.1+195.0,-4.0,b,None,None,None


In [16]:
final.shape, final.drop_duplicates().shape, final.fasta_id.nunique()

((389246, 24), (389246, 24), 385921)

In [17]:
# final

In [18]:
final[['fasta_id', 'bitscore']].drop_duplicates().shape

(389246, 2)

In [19]:
counts = final.groupby(['fasta_id', 'subunit']).agg(lambda df: len(df))

In [20]:
(counts.start == 2).sum()

0

In [21]:
final.opt = final.opt.apply(lambda x: None if (not x is None) and (len(x) == 1) else x)
final.reset_index(inplace = True)

In [22]:
def set_N(final, df, t = 'N'):
        h = hash(tuple(df[df.subunit == 'alpha'].iloc[0]))
        final.at[df.index, 'enz']  = h
        final.at[df.index, 'enzt'] = 'N'
        final.at[df.index, 'step'] = t

for (genome, seq, o), df in tqdm(final.groupby(['genome', 'seq_id', 'operons'])):
    if df.opt.iloc[0] == 'BEacbDAG':
        if len(df[df.subunit.isin(['b', 'delta'])].fasta_id.unique()) == 1:
            set_N(final, df,t = 'N_bdelta')
    if df.opt.iloc[0] == 'BEacDbAG':
        if len(df[df.subunit.isin(['b', 'delta'])].fasta_id.unique()) == 1:
            set_N(final, df, t = 'N_bdelta')
    if df.opt.iloc[0] == 'BEacbAG':
            set_N(final, df, t = 'N_nodelta')

100%|████████████████████████████████████████████████████████████████████████████████| 116956/116956 [00:26<00:00, 4456.60it/s]


In [23]:
final[['opt', 'enzt', 'enz']].drop_duplicates()[['opt', 'enzt',]].value_counts()

opt       enzt
BEacDbAG  N       461
BEacbAG   N       442
BEacbDAG  N       397
dtype: int64

In [24]:
all_subunits = ['alpha', 'beta', 'gamma', 'delta', 'epsilon', 'a', 'b', 'c']
def find0(df):
    x = df.subunit.to_list()
    if len(set(all_subunits) & set(x)) == 8 :
        h = hash(tuple(df[df.subunit == 'alpha'].iloc[0]))
#         global final
        final.at[df.index, 'enz']  = h
        final.at[df.index, 'enzt'] = 'single_operon' 
        final.at[df.index, 'step'] = 'single_operon' 

final[final.enz.isna()].groupby(['genome', 'seq_id', 'operons']).progress_apply(find0)

100%|█████████████████████████████████████████████████████████████████████████████████| 115656/115656 [03:26<00:00, 560.30it/s]


""


In [25]:
final[['enz', 'enzt']].drop_duplicates().enzt.value_counts()

single_operon    19439
N                 1300
Name: enzt, dtype: int64

In [26]:
for (genome, seq, o), df in tqdm(final[final.enz.isna()].groupby(['genome', 'seq_id', 'operons'])):
    x = df.subunit.to_list()
    if len(set(['alpha', 'beta', 'gamma', 'delta', 'a', 'b', 'c']) & set(x)) == 7 :
        if 'epsilon' in final[(final.genome == genome) & (final.seq_id == seq)].subunit.to_list():
            h = hash(tuple(df[df.subunit == 'alpha'].iloc[0]))
            final.at[df.index, 'enz']  = h
            final.at[df.index, 'enzt'] = 'epsilon_away'
            final.at[df.index, 'step'] = 'epsilon_away'

100%|██████████████████████████████████████████████████████████████████████████████████| 96217/96217 [00:54<00:00, 1769.05it/s]


In [27]:
final[['enz', 'enzt']].drop_duplicates().enzt.value_counts()

single_operon    19439
N                 1300
epsilon_away       452
Name: enzt, dtype: int64

In [28]:
from collections import Counter
def clean_ind(final, index):
    # index magic
    index_values = final[index].index
    df = final[index].copy()
    # find duplicates
    counter = Counter(df.subunit.to_list())
    duplicates = [subunit for subunit, c in counter.items() if c > 1]
    if len(duplicates) == 0:
        return index
    
    dup_ind_v   = df[ df.subunit.isin(duplicates)].index
    else_ind_v  = df[~df.subunit.isin(duplicates)].index
    
    
    # score duplicates
    df.at[dup_ind_v, 'score'] = 0
    
    else_seqs = df.loc[else_ind_v].seq_id.unique()
    df.at[dup_ind_v, 'score'] = df.loc[dup_ind_v, 'score'] + \
            df.loc[dup_ind_v].seq_id.apply(lambda x: int(x in else_seqs))
    
    else_operons =  df.loc[else_ind_v].operons.unique()
    df.at[dup_ind_v, 'else_operons'] = df.loc[dup_ind_v].operons.apply(lambda x: x in else_operons)
    
    # find max
    inferred = []
    for _, minidf in df.loc[dup_ind_v].groupby('subunit'):
        inferred.append(minidf.score.idxmax()) 
        inferred += list(minidf[minidf.else_operons].index)
    inferred = list(set(inferred))
        
    # index magic
    inferred = pd.Index(inferred)
    r_v = else_ind_v.union(inferred)
    r = final.index.isin(r_v)
    return r

### test clean df 
### GB_GCA_011048855.1  GB_GCA_002434045.1
# ind = final.genome == 'GB_GCA_002434045.1'
# final[clean_ind(final, ind)]

In [29]:
for genome, df in tqdm(final[final.enz.isna()].groupby(['genome'])):
    operons = df.opt.to_list()
    combinations = [
        ['acb', 'DAGBE'], 
        ['acb', 'DAG', 'BE'],
        ['acb', 'DA', 'GBE'],
        ['acbb', 'DAGBE'], 
        ['acbb', 'DAG', 'BE'],
        ['acbb', 'DA', 'GBE'],
        ['acbD', 'AGBE'],
        ['acbD', 'AG', 'BE'], 
        ['acbDA', 'GBE'], 
        ['acbDAG', 'BE'],       
        ['acbA', 'DGBE'],
        ['acbA', 'DG', 'BE'],
        ['ac', 'bDAGBE'],
        ['ac', 'bDAG', 'BE'],
        ['acD', 'bAGBE',],
        ['acD', 'bAG','BE',],
        ['acb', 'bDAG', 'BE'], 
        ['acbDA', 'DAG', 'BE'],
        ['bacbDAG', 'BE'],
        ['acbDGA', 'BE'],
        ['acbDGAG', 'BE'],
        ['acb', 'bDAG', 'BE'],
        ['acbDAAG', 'BE'],
        ['acbDDAG', 'BE'],
        ['acbDDA', 'GBE'],
        ['acbD', 'AGGBE'],
        ['acbDA', 'GGBE'],
        ['acbDA', 'AG', 'BE'],
        ['acbDD',  'AGBE'],
        ['aacbD', 'AG', 'GBE'],
        ['ac', 'bbDAGBE'],
        ['acbDADG', 'BE'],
        ['acbbDAG', 'BE'],
        ['ac', 'bbDAG', 'BE'],
        ['acbDA', 'AGB', 'BE'],
        
    ]
    for comb in combinations:
        if set(comb) & set(operons)==set(comb):
            ind = (final.index.isin(df.index)) & (final.opt.isin(comb))
            ind = clean_ind(final, ind)

            try:
                h = hash(tuple(final[ind][final[ind].subunit == 'alpha'].iloc[0]))
                final.at[ind, 'enz'] = h
                final.at[ind, 'enzt'] = '_'.join(comb)
                final.at[ind, 'step'] = 'w_op'
            except:
                display(df[ind])
        

100%|███████████████████████████████████████████████████████████████████████████████████| 38588/38588 [06:04<00:00, 105.94it/s]


In [30]:
final.enz.nunique()

31663

In [31]:
# # 33568 to 33569 (+1)
# final.enz.nunique()

In [32]:
for (genome), df in tqdm(final[final.enz.isna()].groupby(['genome'])):
    operons = df.opt.to_list()
    subunits = df[df.opt.isna()].subunit.to_list()
    
    combinations = [  
        [['acbDAG'],          ['beta', 'epsilon']], 
        [['acbDA', 'BE'],     ['gamma']],
        [['acbbDA', 'BE'],    ['gamma']],        
        [['ac', 'bDA', 'BE'], ['gamma']],
        [['ac', 'GB', 'DA'],  ['b', 'epsilon']],
        [['cbDAGBE'],         ['a']],
        [['acb', 'AG', 'BE'], ['delta']],
        [['acbD', 'AGB'],     ['b', 'epsilon']],
        [['acbDG', 'BE'],     ['alpha']],
        [['DAGB', 'acb'],     ['epsilon']],
        [['DGBE', 'acb'],     ['alpha']],
        [['acbDA', 'GB'],     ['epsilon']],
        [['acb','AGB'],       ['delta', 'epsilon']],
        [['cbDAG','BE'],      ['a']],
        [['acbD'],            ['alpha', 'beta', 'gamma', 'epsilon']],
        [['cbDA', 'BE'],      ['gamma', 'a']],
        [['acb', 'BE'],       ['alpha', 'gamma', 'delta']],
        [['cb', 'DAG', 'BE'], ['a']],
        [['acbDAGB'],         ['epsilon']],
        [['acb', 'AGBE'],     ['delta']],
        [['acb', 'DA', 'GB'], ['epsilon']],
        [['acbD', 'GBE'],     ['alpha']],
        [['cbDA', ],          ['a', 'gamma', 'epsilon', 'beta']],
        [['ac','DA', 'BE' ],  ['gamma', 'b']],
        [['acbD', 'BE' ],     ['alpha', 'gamma',]],
        [['acbD', 'AG' ],     ['beta', 'epsilon']],
        [['acb', 'DAG' ],     ['beta', 'epsilon',]],
        [['cbDA', 'GBE' ],    ['a',]],
        [['ac', 'bDA', 'GB'], ['epsilon',]],
        [['acbDA' ],          ['gamma', 'beta', 'epsilon',]],
        [['ac', 'bD', 'AG'], ['beta', 'epsilon']],
        [['acbDA', 'AG', ], ['beta', 'epsilon']],
        [['cbD', 'GG',  'BE'], ['a', 'alpha']],
        [['cb', 'DA'], ['a', 'gamma', 'beta', 'epsilon']],
        [['acb', 'DA'], ['gamma', 'beta', 'epsilon']],
        [['abDAG'], ['c', 'beta', 'epsilon']],
        [['acbDDAGG'], ['epsilon']],
        [['acDbA', 'AG', 'BE'], ['b']],
        [['acDbAG',],[ 'beta', 'epsilon']],
        [['acD', 'AG', 'BE'], ['b']],
        [['bbDAGBE'], ['a', 'c']],
        
                    ]
    
    for comb in combinations:
        if set(comb[0])&set(operons)==set(comb[0]) and set(comb[1])&set(subunits)==set(comb[1]):
            
            ind = (final.index.isin(df.index)) \
                & ((final.opt.isin(comb[0])) | \
                       (final.subunit.isin(comb[1]) & final.opt.isna())
                  )
            ind = clean_ind(final, ind)
            
            h = hash(tuple(final[ind][final[ind].subunit == 'alpha'].iloc[0]))
            final.at[ind, 'enz'] = h
            final.at[ind, 'enzt'] = '_'.join(comb[0] + comb[1])
            final.at[ind, 'step'] = 'op+subs'
    

100%|███████████████████████████████████████████████████████████████████████████████████| 33812/33812 [01:09<00:00, 483.37it/s]


In [33]:
final.enz.nunique()

32611

In [34]:
# 35503 into 35503 (0)
# final.enz.nunique()

In [35]:
## 
## eg: a bDAGBE cbAGBE
for (genome), df in tqdm(final[final.enz.isna()].groupby(['genome'])):
    if not len(set(df.subunit.to_list())&set(all_subunits)) == 8:
        continue
    if len(df) > 14: # один из фильтров на 2 фермента
        continue
        
    ind = (final.index.isin(df.index)) 
    ind = clean_ind(final, ind) # зафейлит на a,bDAGBE,cbAGBE
    
    counter = Counter(final[ind].subunit.to_list())
    if counter['alpha'] > 1 and counter['beta'] > 1 and ((counter['gamma'] > 1) or (counter['epsilon'] > 1)):
        continue
    
    operons  = final[ind][['opt', 'operons']].drop_duplicates().opt.dropna().to_list()
    subunits = final[ind][final[ind].opt.isna()].subunit.to_list()
   
    name = operons + subunits
    sort_key = {'a' : 0,  'b': 1,  'c' : 3,
                'D' : 4,  'delta':5,
                'A':6,    'alpha':7, 
                'G':8,    'gamma':9,
                'B' :10,  'beta':11,  
                'E':12,   'epsilon':13 }

    name = sorted(name, key = lambda x: sort_key[x] if x in all_subunits else sort_key[x[0]])
    
    h = hash(tuple(final[ind][final[ind].subunit.isin(['alpha', 'beta'])].iloc[0]))
    final.at[ind, 'enz'] = h
    final.at[ind, 'enzt'] = '_'.join(name)
    final.at[ind, 'step'] = 'inf_whole'
#     print('_'.join(name))


100%|███████████████████████████████████████████████████████████████████████████████████| 33504/33504 [00:38<00:00, 863.75it/s]


In [36]:
final.enz.nunique()

33976

In [37]:
# # 37945 into 37945 (0)
# final.enz.nunique()

In [38]:
# needs to run two times

for (genome), df in tqdm(final[final.enz.isna()].groupby(['genome'])):
    combinations = [ 

        [['BEacbD'], ['gamma']], #no_alpha
        [['acbDAG'], ['epsilon']], #no_beta
        [['acbDA'], ['gamma']], #no_beta #no_epsilon
        [['acbDAG'], []], #no_beta #no_epsilon
        [['cbDA', 'BE' ], ['gamma']],#no_a
        [['cbDAG', 'BE'], []], #no_a
        [['bDAGBE'], []], #no_a #no_c
        [['AGBE'], ['b']], #no_a #no_c
        [['abDAG',  'BE'], []],#no_c
        [['bD', 'AG', 'BE'], ['a']], #no_c
        [['ac', 'DA', 'BE'], ['gamma']], #no_b
        [['acb', 'AGBE' ], []], #no_delta
        [['acbDA', 'BE'], []],  #no_gamma 
        [['acbD', 'AG'], ['beta']],  #no_epsilon
        [['acbDAGB'], []], #no_epsilon, 
        [['acbAGBE'], []], #no_delta
        [['AAEcbDGB'], []], #no_a
        [['acbAG'], ['beta', 'epsilon']], #no_delta
        [['cbAGBE'], ['a']], #no_delta 
#         [[], []],    
                    ]
    
    for comb in combinations:
        operons  = df[(df.enz.isna())].opt.dropna().to_list()
        subunits = df[(df.enz.isna()) & (df.opt.isna())].subunit.to_list()
        
        if set(comb[0])&set(operons)==set(comb[0])  and set(comb[1])&set(subunits)==set(comb[1]):
            
            ind = (final.index.isin(df.index)) \
                & (final.opt.isin(comb[0]) | (final.subunit.isin(comb[1]) & final.opt.isna()) )              
            ind = clean_ind(final, ind)

            
            h = hash(tuple(final[ind][final[ind].opt.isin(comb[0])].iloc[0]))
            final.at[ind, 'enz']  = h
            final.at[ind, 'enzt'] = '_' + '_'.join(comb[0] + comb[1])
            final.at[ind, 'step'] = 'partial'
            break
#             print(final[ind])
#             print(~df.index.isin(final[ind].index))
    

100%|████████████████████████████████████████████████████████████████████████████████████| 33042/33042 [09:48<00:00, 56.15it/s]


In [70]:
# final[final.genome == 'RS_GCF_903644075.1']

In [39]:
for (genome), df in tqdm(final[final.enz.isna()].groupby(['genome'])):
# for (genome), df in final[final.seq_id == 'DFZJ01000138.1'].groupby(['genome']):
    if not len(set(df.subunit.to_list())&set(all_subunits)) >= 5:
        continue
    if len(df) > 12:
        continue
    
    ind = (final.index.isin(df.index)) 
    ind = clean_ind(final, ind)
    
    operons  = final[ind][['opt', 'operons']].drop_duplicates().opt.dropna().to_list()
    subunits = final[ind][final[ind].opt.isna()].subunit.to_list()
   
    sort_key = {'a' : 0,  'b': 1,  'c' : 3,
                'D' : 4,  'delta':5,
                'A':6,    'alpha':7, 
                'G':8,    'gamma':9,
                'B' :10,  'beta':11,  
                'E':12,   'epsilon':13 }


    name = operons + subunits
    name = sorted(name, key = lambda x: sort_key[x] if x in all_subunits else sort_key[x[0]])

    
    h = hash(tuple(final[ind].iloc[0]))
    final.at[ind, 'enz'] = h
    final.at[ind, 'enzt'] = '_'+ '_'.join(name)
    final.at[ind, 'step'] = 'inf_partial'

100%|███████████████████████████████████████████████████████████████████████████████████| 32370/32370 [02:09<00:00, 249.85it/s]


In [40]:
final.enz.nunique()

41282

In [41]:
# #42454 into 42461 (+7)
# final.enz.nunique()

In [42]:
final.to_csv('genomes_info_F_enz_v1.csv', index =  False)

In [43]:
final = pd.read_csv('genomes_info_F_enz_v1.csv')

In [44]:
# test NZ_FMYP01000086.1
# final = final[final.seq_id == 'NZ_FMYP01000086.1']

In [45]:
'''
Логика в этом поиске:
некоторые N ферменты оказались в разных оперонах из-за расстояния ~250 между генами
Нужно проверить расстояние между оперонами в "генах" и посмотреть 
либо срощение b и delta либо отсутствие delta
'''

for _, df in final.groupby('enz'):
    df = df[df.enzt != 'N']
    subunits = df.subunit.to_list()
    if not set(subunits) == set(['alpha', 'beta', 'gamma', 'delta', 'epsilon', 'a', 'b', 'c']):
        continue
    if not df.seq_id.nunique() == 1:
        continue
        
    if len(df[df.subunit.isin(['b', 'delta'])].fasta_id.unique()) == 1:
        ep_N = df[df.subunit == 'epsilon'].N.iloc[0]
        a_N  = df[df.subunit == 'a'      ].N.iloc[0]
        if abs(ep_N-a_N) == 3:
            final.at[df.index, 'enzt'] = 'N'
            final.at[df.index, 'step'] = df.step.iloc[0] + '_N_bdelta'

In [46]:
for _, df in final.groupby('enz'):
    df = df[df.enzt != 'N']
    subunits = df.subunit.to_list()
    if not set(subunits) == set(['alpha', 'beta', 'gamma', 'epsilon', 'a', 'b', 'c']):
        continue
    if not df.seq_id.nunique() == 1:
        continue    
    ep_N = df[df.subunit == 'epsilon'].N.iloc[0]
    a_N  = df[df.subunit == 'a'      ].N.iloc[0]
    if abs(ep_N-a_N) == 3:
        final.at[df.index, 'enzt'] = 'N'
        final.at[df.index, 'step'] = df.step.iloc[0] + '_N_nodelta'

In [47]:
#N test seq_id == 'NZ_FMYP01000086.1 NZ_FMWO01000048.1 'DFZJ01000138.1'
# final[final.seq_id == 'DFZJ01000138.1']

In [122]:
final['enzFN'] = final.enzt.apply(lambda x: 'N' if x == 'N' else 'F')

In [49]:
final.to_csv('genomes_info_F_enz_v2.csv', index=  False)

# check double cats in enzymes

In [144]:
final = pd.read_csv('genomes_info_F_enz_v2.csv')

In [145]:
cols = [ 'bitscore','logevalue',  'subunit', 'strand', 'ID', 'N','operons', 'enz', 'opt', 'enzt', 'step','enzFN','fasta_id']

In [149]:
normal = ['_acbb_DAAGBB', 'BBEacbDAAG', 'acbDAAGBBE', 'BBEacbDAAG', '_acbb_DAAGBB']

final_version = final[~final.opt .isin(normal)]
final_version = final_version[~final_version.enzt.isin(normal)]

alpha_in_enzyme = final_version.groupby('enz').apply(lambda x: (x.subunit == 'alpha').sum())
beta_in_enzyme  = final_version.groupby('enz').apply(lambda x: (x.subunit == 'beta').sum())
# venn2([
#     set(alpha_in_enzyme[alpha_in_enzyme == 2].index), 
#     set(beta_in_enzyme [beta_in_enzyme == 2].index), 
# ])

In [150]:
double = set(alpha_in_enzyme[alpha_in_enzyme == 2].index) & set(beta_in_enzyme[beta_in_enzyme == 2].index)
print(len(double))

0


In [142]:
# for en in double:
#     print(final[final.enz == en].enzt.unique())    

In [143]:
# n = 1
# final[final.genome == final[final.enz == list(double)[n]].genome.iloc[0]][cols + ['genome']]\
# .sort_values(['enz'])

In [120]:
# final[final.genome == 'GB_GCA_012719135.1'][cols]

In [148]:
ind = final.ID.isin(['467_1', '467_2', '467_2', '467_4', '467_5', '142_14', '142_15'])
final.at[ind, 'enzt']  = '_acbDA_BE'; final.at[ind, 'enz']  = 5; 


ind = ((final.operons == 'JAAYAW010000079.1-3.0') & (final.N.isin([40, 41, 42, 43, 44, 45, 46, 47])))
final.at[ind, 'enzt']  = 'N'; final.at[ind, 'enzFN']  = 'N'; final.at[ind, 'enz']  = 4; 

ind = ((final.operons == 'JAAYAW010000079.1-3.0') & (final.N.isin([29, 30, 31, 32, 33, 34, 35, 36])))
final.at[ind, 'opt']  = 'acbDAGBE';


ind = (final.operons == 'MFPH01000013.1+0.0')
final.at[ind, 'enzt']  = '_DAGBE'; final.at[ind, 'enz']  = 1

ind = (final.operons == 'MFPH01000034.1-0.0')
final.at[ind, 'enzt']  = '_cbAGBE'; 

ind = (final.operons == 'NZ_FNPJ01000029.1+0.0')
final.at[ind, 'enzt']  = '_acbDAGB';

ind = (final.operons == 'NZ_FNPJ01000081.1+1.0')
final.at[ind, 'enzt']  = 'N'; final.at[ind, 'enzFN']  = 'N'; final.at[ind, 'enz']  = 2;

ind = (final.operons == 'DIZP01000011.1-1.0')
final.at[ind, 'enzt']  = '_a_bDAGB'; 

ind = (final.operons == 'DIZP01000037.1+1.0') | (final.fasta_id == 'DIZP01000003.1_4')
final.at[ind, 'enzt']  = '_DAGBE'; final.at[ind, 'enz']  = 3; 

# ind = (final.seq_id == 'JAAYAW010000079.1') & (final.N.isin([47, 46, 44, 43, 42, 41, 40]))
# final.at[ind, 'enzt'] = 'N'; final.at[ind, 'enz']  = 'a'

# ind = (final.seq_id == 'NZ_FNPJ01000081.1') & (final.N.isin([6, 7, 9, 10, 11, 12]))
# final.at[ind, 'enzt'] = 'N'; final.at[ind, 'enz']  = 'b'; final.at[ind, 'step']  = 'manual_N'


# ind = (final.seq_id == 'NZ_FNPJ01000081.1') & (final.N.isin([6, 7, 9, 10, 11, 12]))
# final.at[ind, 'enzt'] = 'N'; final.at[ind, 'enz']  = 'c'; final.at[ind, 'step']  = 'manual_N'

In [151]:
final.to_csv('genomes_info_F_enz_v3.csv', index=  False)

In [54]:
# final[final.genome == 'GB_GCA_012719135.1'][cols]

In [56]:
# tmp = final[['genome', 'seq_id', 'enzt', 'enz', 'opt']].drop_duplicates()

In [57]:
# tmp[tmp .enz.isin(list(double))].sort_values('enzt').groupby('genome').apply(lambda x: display(x))

In [58]:
# final[final.genome == 'RS_GCF_000614065.1'][cols]

In [59]:
# ! grep GB_GCA_002423025 ../gtdb_info/gtdb_taxonomy.csv

In [60]:
# final[final.enz.isin(double)].enzt.unique()

# Check unassigned subunits

In [61]:
# final = pd.read_csv('genomes_info_F_enz.csv')

In [62]:
l = []
for i, df in final[final.enz.isna()].groupby('genome'):
    vc = dict(df.subunit.value_counts())
    vc['genome'] = i
    l.append(vc)
    
noncanonical = pd.DataFrame(l)
noncanonical = noncanonical.set_index('genome')
noncanonical[['alpha', 'beta']].notna().all(axis = 1).sum()

435

In [63]:
noncanonical[['alpha', 'beta', 'c']].notna().all(axis = 1).sum()

143

In [64]:
noncanonical[['alpha', 'beta', 'c', 'a']].notna().all(axis = 1).sum()

121

In [65]:
noncanonical[noncanonical[['alpha', 'beta', 'c', 'a', 'b', 'delta', 'gamma', 'epsilon']]\
             .notna().all(axis = 1)].head(10)
# .sort_values('alpha', ascending = False)\

,b,c,alpha,beta,delta,gamma,epsilon,a
genome,,,,,,,,
GB_GCA_001303025.1,4.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0
GB_GCA_001304425.1,3.0,3.0,1.0,2.0,3.0,2.0,3.0,3.0
GB_GCA_001314685.1,3.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0
GB_GCA_001657325.1,5.0,2.0,1.0,1.0,3.0,1.0,2.0,2.0
GB_GCA_001789745.1,2.0,2.0,1.0,2.0,3.0,2.0,2.0,1.0
GB_GCA_001797845.1,5.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0
GB_GCA_001873935.1,4.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
GB_GCA_002327415.1,3.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0
GB_GCA_002335355.1,4.0,1.0,2.0,3.0,2.0,2.0,1.0,2.0


In [66]:
# final[final.genome == 'GB_GCA_001304425.1'].sort_values(['enz', 'operons'])[cols]

In [67]:
# s = 'a'
# dfs[s][dfs[s].genome == 'GB_GCA_002691565.1']

In [68]:
# final[['enz', 'enzt']].drop_duplicates().enzt.value_counts()[:40]